In [1]:
import pandas as pd
import numpy as np
import mysql.connector
import os
from mysql.connector import Error
from datetime import datetime, date, timedelta
import re
import ConeccionesSucursales as cs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
#Definimos la ruta en donde queremos guardar los archivos
ruta = '/home/estadistico/Documents/Erick/'
ruta_guardar = '/home/estadistico/Documents/Erick/Reportes diarios'
mes = 20210101
mes2 = 'Enero2021'
mes3 = pd.to_datetime('01/01/2021',dayfirst=True)
fin_mes = pd.to_datetime('31/01/2021',dayfirst=True)

In [3]:
#Lectura pagos Credifiel
book = os.listdir(''+ruta+'/'+str(mes2)+'/Credifiel/Pagos/')
pagos_cred = pd.DataFrame()
for j in book:
    pagos_new = pd.read_excel(''+ruta+'/'+str(mes2)+'/Credifiel/Pagos/'+j+'',converters={'ID':str})
    pagos_cred = pd.concat([pagos_cred,pagos_new],ignore_index=True)
    pagos_cred = pagos_cred.drop_duplicates()
#pagos_cred = pagos_cred.iloc[:,[12,1,8]]
pagos_cred = pd.pivot_table(pagos_cred,index=['ID','FECHA'],values=['IMPORTE'],aggfunc=np.sum)
pagos_cred = pd.DataFrame(pagos_cred.to_records())
pagos_cred = pagos_cred.loc[pagos_cred['FECHA']>=mes3]

In [4]:
#Lectura asignacion Credifiel
sheet_cred2 = os.listdir(''+ruta+'/'+str(mes2)+'/Credifiel/Asignacion/')
asignacion_cred = pd.DataFrame()
for j in sheet_cred2:
    asignacion_cred_new = pd.read_excel(''+ruta+'/'+str(mes2)+'/Credifiel/Asignacion/'+j+'',converters={'CREDITO':str})
    asignacion_cred = pd.concat([asignacion_cred,asignacion_cred_new],ignore_index=True)
    asignacion_cred = asignacion_cred.drop_duplicates(subset='CREDITO')
asignacion_cred = asignacion_cred.loc[:,['CREDITO','ARANCEL']]
asignacion_cred.columns = ['credito','arancel']
asignacion_cred.head(2)

,credito,arancel
0,9,0.4
1,831,0.4


In [5]:
#Asignamos valores a los parametros \n"
today = date.today().strftime('%Y%m%d')
now = datetime.now().strftime('%d-%m-%Y %H:%M')
hoy = np.datetime64('today')
ayer = np.datetime64(date.today() - timedelta(1))
today2 = date.today().strftime('%d-%m-%Y')

In [6]:
#Funcion para transformar varias columnas a numero
def transform_numeric(df,cols):
    return df[cols].apply(pd.to_numeric, errors='coerce')

#Funcion que convierte a dataframe y rellena espacios con 0
def to_dataframe(df):
    df = pd.DataFrame(df.to_records())
    df.fillna(0,inplace=True)
    return df

#Funcion que quita el % del string
def percent_tonumeric(df,cols):
    for c in cols:
        df[c] = [x[:-2] for x in df[c]]
    return transform_numeric(df,cols)

# Llamadas detallado Credifiel

In [7]:
#Hacemos la consulta referente a las gestiones de Credifiel
llamadas_cred = cs.query_naucalpan('procesos_externos.all_rpt_llamadas_detallado',mes,today,'(^CRED)')
llamadas_cred.columns = ['server_ip','uniqueid','call_type','call_date','campaign_id','campaign_name','list_id','list_name','lead_id','vendor_lead_id','vendor_lead_name','phone_number','status','status_name','username','full_name','term_reason_call','queue_seconds','length_in_sec','call_date_short','call_time_short','call_time_hour','call_range_hour','dialstatus','hangup_cause','sip_hangup_cause','carrier','areacode','state','region']
llamadas_cred.head(1)

,server_ip,uniqueid,call_type,call_date,campaign_id,campaign_name,list_id,list_name,lead_id,vendor_lead_id,...,call_time_short,call_time_hour,call_range_hour,dialstatus,hangup_cause,sip_hangup_cause,carrier,areacode,state,region
0,192.168.15.21,1609596926.193234,Outbound,2021-01-02 08:15:26,1301,CREDIFIEL,10477,0912_PRED_CRF_NUEVAS,29537153,17_1_CR-317856,...,0 days 08:15:26,8,08:00 - 09:00,CONGESTION,34,503 Service Unavailable,CONVERGIA,993,TAB,SURESTE


In [8]:
llamadas_cred['turno'] = np.where(llamadas_cred['call_time_hour']<=14,'M','V')

In [9]:
#Obtenemos el performance bruto de cada gestor
control_gestores_bruto_cred = pd.pivot_table(llamadas_cred,index=['full_name','turno'],values=['length_in_sec'],aggfunc=[np.sum,np.mean,'count'])
control_gestores_bruto_cred = to_dataframe(control_gestores_bruto_cred)
control_gestores_bruto_cred = control_gestores_bruto_cred.iloc[:,[0,1,4,2,3]]
control_gestores_bruto_cred.columns = ['Gestor','Turno','NumeroGestionesB','TiempoTotalSegB','PromedioLlamadaSegB']
control_gestores_bruto_cred['TiempoTotalMinB'] = [x/60 for x in control_gestores_bruto_cred['TiempoTotalSegB']]

In [10]:
#Obtenemos el performance de las llamadas contestadas
llamadas_cred2 = llamadas_cred.loc[llamadas_cred['status_name'].isin(['Contacto con titular','Promesa de pago','Mensaje con terceros','Mensaje con familiar','Negativa de pago','Contacto familiar','Agendar llamada'])]
control_gestores_llamadas_cred = pd.pivot_table(llamadas_cred2,index=['full_name','turno'],values=['length_in_sec'],aggfunc=[np.sum,np.mean,'count'])
control_gestores_llamadas_cred = to_dataframe(control_gestores_llamadas_cred)
control_gestores_llamadas_cred = control_gestores_llamadas_cred.iloc[:,[0,1,4,2,3]]
control_gestores_llamadas_cred.columns = ['Gestor','Turno','NumeroGestiones','TiempoTotalSeg','PromedioLlamadaSeg']
control_gestores_llamadas_cred['TiempoTotalMin'] = [x/60 for x in control_gestores_llamadas_cred['TiempoTotalSeg']]

In [11]:
control_cred_turno = pd.merge(control_gestores_bruto_cred,control_gestores_llamadas_cred,how='left',on=['Gestor','Turno'])
control_cred_turno.fillna(0,inplace=True)
control_cred_turno.head(2)

,Gestor,Turno,NumeroGestionesB,TiempoTotalSegB,PromedioLlamadaSegB,TiempoTotalMinB,NumeroGestiones,TiempoTotalSeg,PromedioLlamadaSeg,TiempoTotalMin
0,AGUILAR MACEDO VALERIA,M,1,4,4.000000,0.066667,0.0,0.0,0.0,0.000000
1,AGUILAR RIOS DANIEL ALEJANDRO,M,296,3865,13.057432,64.416667,1.0,8.0,8.0,0.133333


In [12]:
#Obtenemos el performance de las llamadas contestadas
control_gestores_total_cred = pd.pivot_table(llamadas_cred2,index=['full_name'],values=['length_in_sec'],aggfunc=[np.sum,np.mean,'count'])
control_gestores_total_cred = to_dataframe(control_gestores_total_cred)
control_gestores_total_cred = control_gestores_total_cred.iloc[:,[0,3,1,2]]
control_gestores_total_cred.columns = ['Gestor','NumeroGestiones','TiempoTotalSeg','PromedioLlamadaSeg']
control_gestores_total_cred['TiempoTotalMin'] = [x/60 for x in control_gestores_total_cred['TiempoTotalSeg']]
control_gestores_total_cred

,Gestor,NumeroGestiones,TiempoTotalSeg,PromedioLlamadaSeg,TiempoTotalMin
0,AGUILAR RIOS DANIEL ALEJANDRO,1,8,8.000000,0.133333
1,BALBUENA MACEDO EDGAR URIEL,1,439,439.000000,7.316667
2,CASTILLEJO CASTILLEJO DULCE MICHELLE,3,205,68.333333,3.416667
3,GARCIA CHAVEZ BRAYAN,36,5442,151.166667,90.700000
4,HERNANDEZ MEDEL LUIS ENRIQUE,2,62,31.000000,1.033333
5,HERNANDEZ VELASCO YALITZI AZAYAEEL,43,16862,392.139535,281.033333
6,HUERTA LOPEZ NATALIA XOCHITL,1,222,222.000000,3.700000
7,JUAREZ ROBLEDO LORENA,1,37,37.000000,0.616667
8,LOPEZ ESTRADA MA. IGNACIA,2,149,74.500000,2.483333
9,LUCIA ESMERALDA LONGORIA CISNEROS,32,5197,162.406250,86.616667


# Performance Credifiel

In [13]:
#Hacemos la consulta referente al performance del Gestor
performance_all = cs.query_naucalpan('procesos_externos.all_rpt_agent_performance',mes,today)
performance_all.columns = ['fecha','usuario','nombre_usuario','user_group','llamadas','aht','ocupacion','adherencia','login_vici','logout_vici','staff_time','wait_time','talk_time','dispo_time','pause','login','manual','gestion','agua','alimentos','sanitario','retroalimentacion','tranferencia','validacion','whatsapp','incumplimiento','recordatorios','capacitacion','sistemas','lagged','tipificando','*']

In [14]:
performance_cred = performance_all.loc[performance_all['user_group']=='AFINES']
performance_cred[['ocupacion','adherencia']] = percent_tonumeric(performance_cred,['ocupacion','adherencia'])

In [15]:
performance_cred['hora_login'] = [int(x[:2]) for x in performance_cred['login_vici']]
performance_cred['hora_logout'] = [int(x[:2]) for x in performance_cred['logout_vici']]
performance_cred['turno'] = np.where((performance_cred['hora_logout']<=14)&(performance_cred['hora_logout']-performance_cred['hora_login']<=8),'M',
                                    np.where((performance_cred['hora_logout']>17)&(performance_cred['hora_logout']-performance_cred['hora_login']<=8),'V','Mixto'))

In [16]:
performance_cred['horas_trabajadas'] = performance_cred['hora_logout']-performance_cred['hora_login']
performance_cred['llamadas_hora'] = performance_cred['llamadas']/performance_cred['horas_trabajadas']
performance_cred['llamadas_hora'] = performance_cred['llamadas_hora'].round(1)
performance_cred.head(1)

,fecha,usuario,nombre_usuario,user_group,llamadas,aht,ocupacion,adherencia,login_vici,logout_vici,...,capacitacion,sistemas,lagged,tipificando,*,hora_login,hora_logout,turno,horas_trabajadas,llamadas_hora
50,2021-01-02,AGUILARRIAL,AGUILAR RIOS DANIEL ALEJANDRO,AFINES,354,00:00:32,24.23,88.12,08:05:59,14:00:01,...,00:00:11,00:00:00,00:00:00,00:08:31,00:24:05,8,14,M,6,59.0


In [17]:
control_performance_cred_turno = pd.pivot_table(performance_cred,index=['nombre_usuario','turno'],values=['llamadas','ocupacion','llamadas_hora'],aggfunc=[np.sum,np.mean,'count',min,max])
control_performance_cred_turno = to_dataframe(control_performance_cred_turno)
control_performance_cred_turno = control_performance_cred_turno.iloc[:,[0,1,2,3,4,5,6,7,8,11,12,13,14,15,16]]
control_performance_cred_turno.columns = ['Gestor','Turno','SumLlamadasDia','SumLlamadasHora','SumOcupacion','PromedioLlamadasDia','PromedioLlamadasHora','PromedioOcupacion','DiasAsitencia','MinLlamadasDia','MinLlamadasHora','MinOcupacion','MaxLlamadasDia','MaxLlamadasHora','MaxOcupacion']
control_performance_cred_turno.head(1)

,Gestor,Turno,SumLlamadasDia,SumLlamadasHora,SumOcupacion,PromedioLlamadasDia,PromedioLlamadasHora,PromedioOcupacion,DiasAsitencia,MinLlamadasDia,MinLlamadasHora,MinOcupacion,MaxLlamadasDia,MaxLlamadasHora,MaxOcupacion
0,AGUILAR RIOS DANIEL ALEJANDRO,M,4434,646.6,294.86,341.076923,49.738462,22.681538,13,198,33.0,18.21,460,65.7,30.06


In [18]:
control_performance_cred_turno2 = control_performance_cred_turno[['Gestor','Turno','PromedioLlamadasHora','PromedioOcupacion']]

In [19]:
control_performance_cred = pd.pivot_table(performance_cred,index=['nombre_usuario'],values=['llamadas','ocupacion','llamadas_hora','horas_trabajadas'],aggfunc=[np.sum,np.mean,'count',min,max])
control_performance_cred = to_dataframe(control_performance_cred)
control_performance_cred = control_performance_cred.iloc[:,[0,1,2,6,7,8]]
control_performance_cred.columns = ['Gestor','SumHorasTrabajadas','SumLlamadasDia','PromedioLlamadasDia','PromedioLlamadasHora','PromedioOcupacion']
control_performance_cred.head(3)

,Gestor,SumHorasTrabajadas,SumLlamadasDia,PromedioLlamadasDia,PromedioLlamadasHora,PromedioOcupacion
0,AGUILAR RIOS DANIEL ALEJANDRO,94,4659,332.785714,49.400000,23.785000
1,CASTILLEJO CASTILLEJO DULCE MICHELLE,121,3784,236.500000,31.425000,43.106250
2,GARCIA CHAVEZ BRAYAN,98,1326,94.714286,13.535714,30.282857


In [20]:
conjunto_performance_llamadas_cred = pd.merge(control_gestores_total_cred,control_performance_cred,on='Gestor',how='left')
conjunto_performance_llamadas_cred['ContactoHora'] = conjunto_performance_llamadas_cred['NumeroGestiones']/conjunto_performance_llamadas_cred['SumHorasTrabajadas']
conjunto_performance_llamadas_cred['PromedioLlamadasHora'] = conjunto_performance_llamadas_cred['SumLlamadasDia']/conjunto_performance_llamadas_cred['SumHorasTrabajadas']
conjunto_performance_llamadas_cred = conjunto_performance_llamadas_cred.iloc[:,[0,8,10,9,]]
conjunto_performance_llamadas_cred = conjunto_performance_llamadas_cred.dropna()
conjunto_performance_llamadas_cred

,Gestor,PromedioLlamadasHora,ContactoHora,PromedioOcupacion
0,AGUILAR RIOS DANIEL ALEJANDRO,49.563830,0.010638,23.785000
2,CASTILLEJO CASTILLEJO DULCE MICHELLE,31.272727,0.024793,43.106250
3,GARCIA CHAVEZ BRAYAN,13.530612,0.367347,30.282857
10,MEDINA ARIAS MAURICIO,31.306452,0.008065,42.416667
11,MISHELLLE ADYANI CHAVEZ CASTRO,24.548913,0.483696,29.928333
13,QUINTOS ARVIZU LUIS DANIEL,52.347826,0.028986,44.399412


# Gestion detallado Credifiel

In [21]:
#Hacemos la consulta referente a las gestiones de Credifiel
gestion_cred = cs.query_naucalpan('credifiel_rpt_gestiones_detallado',mes,today)
gestion_cred.columns = ['folio_gestion','unegocio_id','fecha_gestion','hora_gestion','credito','nombre_credito','telefono','usuario','nombre_usuario','accion','resultado','fecha_promesa','monto_promesa','comentarios','accion_credifiel','resultado_credifiel','dictamen']
gestion_cred['monto_promesa'] = pd.to_numeric(gestion_cred['monto_promesa'])
gestion_cred = gestion_cred.loc[gestion_cred['accion_credifiel']=='PDP']
gestion_cred['division'] = 'Extrajudicial'
gestion_cred['credito'] = [x[3:] for x in gestion_cred['credito']]
gestion_cred.head(1)

,folio_gestion,unegocio_id,fecha_gestion,hora_gestion,credito,nombre_credito,telefono,usuario,nombre_usuario,accion,resultado,fecha_promesa,monto_promesa,comentarios,accion_credifiel,resultado_credifiel,dictamen,division
8,9518854,1,2021-01-02,0 days 10:16:54,380282,HIPOLITO OJEDA DAVID,2741226882,QUINTOSARLD,QUINTOS ARVIZU LUIS DANIEL,LLAMADA A TITULAR,SEGUIMIENTO A PROMESA,,NaN,2741226882 SE HACE RECORDATORIO DE PAGO POR WH...,PDP,Promesa de Pago,CONTACTO,Extrajudicial


In [22]:
#Hacemos el consolidado de la asignacion y promesado
consolidado_cred = pd.merge(gestion_cred,asignacion_cred,how='left',on=['credito'])
cols = ['monto_promesa']
consolidado_cred[cols] = consolidado_cred[cols].apply(pd.to_numeric, errors='coerce')
consolidado_cred = consolidado_cred.loc[consolidado_cred['monto_promesa']>50]
consolidado_cred = consolidado_cred[(consolidado_cred['monto_promesa']<500000)]

In [23]:
# pagos_cred_fact = pd.merge(pagos_cred,asignacion_cred,how='left',left_on=['ID'],right_on=['credito'])
# pagos_cred_fact['arancel'] = np.where(pagos_cred_fact['arancel'].isna(),np.mean(pagos_cred_fact['arancel']),pagos_cred_fact['arancel'])
# consolidado_pagos_cred.head(1)

In [24]:
consolidado_pagos_cred = pd.merge(consolidado_cred,pagos_cred,how='left',left_on=['credito'],right_on=['ID'])
consolidado_pagos_cred['arancel'] = np.where(consolidado_pagos_cred['arancel'].isna(),np.mean(consolidado_pagos_cred['arancel']),consolidado_pagos_cred['arancel'])
consolidado_pagos_cred['fecha_promesa'] = pd.to_datetime(consolidado_pagos_cred['fecha_promesa'],dayfirst=False)

In [25]:
#Condicionamos a los gestores cuyas promesas y pagos estan en un plazo de 3 dias
#consolidado_pagos_cred['PagoUnico'] = [np.where((z < x+pd.Timedelta(4, unit='D')) & ((z > x-pd.Timedelta(4, unit='D') )), y, 0) for x,y,z in zip(consolidado_pagos_cred['fecha_promesa'],consolidado_pagos_cred['IMPORTE'],consolidado_pagos_cred['FECHA'])]
#consolidado_pagos_cred['PagoUnico'] = [x.item() for x in consolidado_pagos_cred['PagoUnico']]

In [26]:
consolidado_pagos_cred['PagoReal'] = [x*y for x,y in zip(consolidado_pagos_cred['IMPORTE'],consolidado_pagos_cred['arancel'])]
consolidado_pagos_cred2 = consolidado_pagos_cred.drop_duplicates(subset=['credito','IMPORTE','FECHA','usuario'])

In [27]:
pagos_totales_cred = pd.pivot_table(consolidado_pagos_cred2,index=['nombre_usuario'],values=['monto_promesa','IMPORTE','PagoReal'],aggfunc=['count',np.sum],dropna=False)
pagos_totales_cred = pd.DataFrame(pagos_totales_cred.to_records())
pagos_totales_cred = pagos_totales_cred.iloc[:,[0,3,1,6,4,5]]
pagos_totales_cred.columns = ['Gestor','NumeroPromesas','NumeroPagos','SumaPromesas','SumaPagos','SumaArancel']
pagos_totales_cred.fillna(0,inplace=True)
pagos_totales_cred['EfVolumen'] = pagos_totales_cred['NumeroPagos']/pagos_totales_cred['NumeroPromesas']
pagos_totales_cred['EfPagos'] = pagos_totales_cred['SumaPagos']/pagos_totales_cred['SumaPromesas']
#pagos_totales_cred['FechaFact'] = max(pagos_brad2['FECHA_PAGO'])
pagos_totales_cred.head(1)

,Gestor,NumeroPromesas,NumeroPagos,SumaPromesas,SumaPagos,SumaArancel,EfVolumen,EfPagos
0,GARCIA CHAVEZ BRAYAN,36,24,154128.68,77224.99,11778.2136,0.666667,0.501042


In [28]:
gestor_final_cred = pagos_totales_cred[['Gestor','SumaPagos','EfPagos']]
gestor_final_cred

,Gestor,SumaPagos,EfPagos
0,GARCIA CHAVEZ BRAYAN,77224.99,0.501042
1,HERNANDEZ VELASCO YALITZI AZAYAEEL,12407.00,0.328941
2,LUCIA ESMERALDA LONGORIA CISNEROS,7320.03,0.220785
3,MISHELLLE ADYANI CHAVEZ CASTRO,33635.00,0.700701
4,ZARZA NORIA JEFREY ALBERTO,2900.00,1.000000


In [29]:
final_cred = pd.merge(gestor_final_cred,conjunto_performance_llamadas_cred,how='outer',on='Gestor')
final_cred['PromedioLlamadasHora'] = final_cred['PromedioLlamadasHora'].round()
final_cred['PromedioOcupacion'] = final_cred['PromedioOcupacion']/100
cols = ['PromedioOcupacion','EfPagos','ContactoHora']
final_cred[cols] = final_cred[cols].round(2)
final_cred.fillna(0,inplace=True)
final_cred

,Gestor,SumaPagos,EfPagos,PromedioLlamadasHora,ContactoHora,PromedioOcupacion
0,GARCIA CHAVEZ BRAYAN,77224.99,0.50,14.0,0.37,0.30
1,HERNANDEZ VELASCO YALITZI AZAYAEEL,12407.00,0.33,0.0,0.00,0.00
2,LUCIA ESMERALDA LONGORIA CISNEROS,7320.03,0.22,0.0,0.00,0.00
3,MISHELLLE ADYANI CHAVEZ CASTRO,33635.00,0.70,25.0,0.48,0.30
4,ZARZA NORIA JEFREY ALBERTO,2900.00,1.00,0.0,0.00,0.00
5,AGUILAR RIOS DANIEL ALEJANDRO,0.00,0.00,50.0,0.01,0.24
6,CASTILLEJO CASTILLEJO DULCE MICHELLE,0.00,0.00,31.0,0.02,0.43
7,MEDINA ARIAS MAURICIO,0.00,0.00,31.0,0.01,0.42
8,QUINTOS ARVIZU LUIS DANIEL,0.00,0.00,52.0,0.03,0.44


In [30]:
writer = pd.ExcelWriter(''+ruta_guardar+'/Gestores/Credifiel/Reporte Gestor Credifiel '+now+'.xlsx',engine='xlsxwriter')
final_cred.to_excel(writer,'BSC',index=False,header=True)
gestor_final_cred.to_excel(writer,'MontoRecuperado',index=False,header=True)
control_cred_turno.to_excel(writer,'LlamadasDetallado',index=False,header=True)
control_performance_cred_turno.to_excel(writer,'PerformanceDetallado',index=False,header=True)
pagos_totales_cred.to_excel(writer,'PagosDetallado',index=False,header=True)

writer.save()